# Set up

In [ ]:
# Install Disimpy

!pip install git+https://github.com/kerkelae/disimpy.git

In [ ]:
# Check available GPU

!nvidia-smi

In [ ]:
# Mount Drive if using Google Colaboratory

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import the required packages and modules

import os
import numpy as np
import matplotlib.pyplot as plt

from disimpy import gradients, simulations, utils

In [ ]:
# Define path and load synthetic axons

path = '/content/drive/MyDrive/final-ufa-model-comparison/ConFiG'

def load_mesh(mesh_file):
    """Load mesh from ply file"""
    with open(mesh_file, 'r') as f:
        mesh_data = f.readlines()
    header = mesh_data[0:mesh_data.index('end_header\n')]
    i = [i for i, e in enumerate(header) if 'element vertex' in e][0]
    if header[i + 1:i + 4] != ['property float x\n', 'property float y\n',
                               'property float z\n']:
        raise Exception(
            'Unsupported mesh file (%s). ' % mesh_file +
            'Vertex positions must be the first vertex positions defined. ' +
            'Please see %s as an example of the supported format.' % (
                os.path.join(os.path.dirname(utils.__file__), 'tests',
                             'example_mesh.ply')))
    n_of_vertices = int([i for i in mesh_data if
                         i.startswith('element vertex ')][0].split()[-1])
    first_vertice_idx = mesh_data.index('end_header\n') + 1
    vertices = np.loadtxt(mesh_data[first_vertice_idx:first_vertice_idx +
                                    n_of_vertices])
    faces = np.loadtxt(mesh_data[first_vertice_idx + n_of_vertices::])[:, 1:4]
    mesh = np.zeros((faces.shape[0], 3, 3))
    for i in range(faces.shape[0]):
        mesh[i, :, :] = vertices[np.array(faces[i], dtype=int)][0:3, 0:3]
    mesh = np.add(mesh, - np.min(np.min(mesh, 0), 0))
    return mesh

mesh = load_mesh(
    os.path.join(path, 'cell_tissue_replicated_periodic.ply')) * 1e-6
mesh_size = np.max(np.max(mesh, 0), 0)
print(mesh_size)
print(mesh.shape)

In [ ]:
# Load and show initial positions

n_s = int(3e6)
pos = np.zeros((n_s, 3))
for i in range(381):
    pos[int(i*5e3):int((i+1)*5e3), :] = np.loadtxt(
        os.path.join(path, 'init_pos_intra_%s.txt' % (i + 1)))
pos[int(1.905e6)::] = np.loadtxt((os.path.join(path, 'init_pos_extra.txt')))

for i in range(382):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    if i < 381:
        ax.scatter(pos[int(i*5e3):int((i+1)*5e3), 0],
                   pos[int(i*5e3):int((i+1)*5e3), 1],
                   pos[int(i*5e3):int((i+1)*5e3), 2], s=1, alpha=.1)
        ax.set_title('Fibre %s' % (i + 1))
    else:
        ax.scatter(pos[int(1.905e6)::, 0],
                   pos[int(1.905e6)::, 1],
                   pos[int(1.905e6)::, 2], s=1, alpha=.1)
    ax.set_xlim([0, 3.9e-5])
    ax.set_ylim([0, 3.9e-5])
    ax.set_zlim([0, 3.2e-5])
    fig.tight_layout()
    plt.show()

In [ ]:
# Define diffusivity

diffusivity = 2e-9

In [ ]:
# Load b-values and b-vectors

bvals = np.loadtxt(os.path.join(path, 'LTE-STE.bval'))[0:107] * 1e6
bvecs = np.round(np.loadtxt(os.path.join(path, 'LTE-STE.bvec'))[:, 0:107], 4).T
bvals[bvals == 0] = 1e-9
bs = np.unique(bvals)
bvals = np.concatenate([bvals, bvals])
bvecs = np.concatenate([bvecs, bvecs], axis=0)
lte_idx = np.arange(0, 107).astype(int)
ste_idx = np.arange(107, 2 * 107).astype(int)
lte_bvals = bvals[lte_idx]
ste_bvals = bvals[ste_idx]

# Simulation 1

In [ ]:
# Define gradient waveform

ste = np.loadtxt(os.path.join(path, 'waveform_STE.txt'))[np.newaxis, :, :]
lte = np.loadtxt(os.path.join(path, 'waveform_LTE.txt'))[np.newaxis, :, :]
plt.plot(lte[0])
plt.show()
plt.plot(ste[0])
plt.show()


# Create gradient array

T = 80e-3
n_t = int(1e4)
gradient = np.concatenate(
    [lte for i in range(107)] + [ste for i in range(107)], axis=0)
dt = T / (gradient.shape[1] - 1)
Rs = [utils.vec2vec_rotmat(np.array([1., 0, 0]), i) if np.linalg.norm(i) != 0 
      else np.eye(3) for i in bvecs]
gradient = gradients.rotate_gradient(gradient, Rs)
gradient, dt = gradients.interpolate_gradient(gradient, dt, n_t)
gradient = gradients.set_b(gradient, dt, bvals)

In [ ]:
# Show an example of diffusion in periodic fibres

N = int(1e3)
fibre_substrate = {'type' : 'mesh',
                   'mesh' : mesh,
                   'initial positions' : pos[0:N],
                   'periodic' : True,
                   'N_sv' : 50}

signals = simulations.simulation(
    N, diffusivity, gradient, dt, fibre_substrate, trajectories='traj.txt')

trajectories = np.loadtxt('traj.txt')
trajectories = trajectories.reshape(
    (trajectories.shape[0], int(trajectories.shape[1] / 3), 3))

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for i in range(trajectories.shape[1]):
    ax.plot(trajectories[:, i, 0],
            trajectories[:, i, 1],
            trajectories[:, i, 2],
            alpha=.5)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_xlim([-np.max(trajectories), np.max(trajectories)])
ax.set_ylim([-np.max(trajectories), np.max(trajectories)])
ax.set_zlim([-np.max(trajectories), np.max(trajectories)])
ax.ticklabel_format(style='sci', scilimits=(0, 0))
plt.show()

In [ ]:
# Run simulation with QTE (in three parts to reduce memory requirements)

substrate = {'type' : 'mesh',
             'mesh' : mesh,
             'initial positions' : pos[0:int(1e6)],
             'periodic' : True,
             'N_sv' : 50}
signals_1 = simulations.simulation(
    int(1e6), diffusivity, gradient, dt, substrate, all_signals=True, seed=1)

substrate = {'type' : 'mesh',
             'mesh' : mesh,
             'initial positions' : pos[int(1e6):int(2e6)],
             'periodic' : True,
             'N_sv' : 50}
signals_2 = simulations.simulation(
    int(1e6), diffusivity, gradient, dt, substrate, all_signals=True, seed=2)

substrate = {'type' : 'mesh',
             'mesh' : mesh,
             'initial positions' : pos[int(2e6)::],
             'periodic' : True,
             'N_sv' : 50}
signals_3 = simulations.simulation(
    int(1e6), diffusivity, gradient, dt, substrate, all_signals=True, seed=3)

np.save(os.path.join(path, 'simulated_signals_1.npy'),
        np.hstack((signals_1, signals_2, signals_3)))

In [ ]:
# Repeat simulation with different random number seeds to quantify variance


for seed in [10, 20, 30, 40]:

    substrate = {'type' : 'mesh',
                'mesh' : mesh,
                'initial positions' : pos[0:int(1e6)],
                'periodic' : True,
                'N_sv' : 50}
    signals_1 = simulations.simulation(
        int(1e6), diffusivity, gradient, dt, substrate, all_signals=True,
        seed=seed + 1)

    substrate = {'type' : 'mesh',
                'mesh' : mesh,
                'initial positions' : pos[int(1e6):int(2e6)],
                'periodic' : True,
                'N_sv' : 50}
    signals_2 = simulations.simulation(
        int(1e6), diffusivity, gradient, dt, substrate, all_signals=True,
        seed=seed + 2)

    substrate = {'type' : 'mesh',
                'mesh' : mesh,
                'initial positions' : pos[int(2e6)::],
                'periodic' : True,
                'N_sv' : 50}
    signals_3 = simulations.simulation(
        int(1e6), diffusivity, gradient, dt, substrate, all_signals=True,
        seed=seed + 3)

    np.save(os.path.join(path, 'simulated_signals_1_seed%s.npy' % seed),
            np.hstack((signals_1, signals_2, signals_3)))

# Simulation 2


In [ ]:
# Define waveforms

lte = np.loadtxt(os.path.join(path, 'waveform_LTE.txt'))
n_zeros = 8
ste = np.zeros((lte.shape[0] * 3 + n_zeros * 2, 3))
ste[0:78, 0] = lte[:, 0] 
ste[78 + n_zeros:78 + n_zeros + 78, 1] = lte[:, 0] 
ste[78 + n_zeros + 78 + n_zeros::, 2] = lte[:, 0] 
ste = ste[np.newaxis, :, :]
lte = np.copy(ste)
lte[:, :, 1::] = 0
plt.plot(lte[0])
plt.show()
plt.plot(ste[0])
plt.show()


# Create gradient array

T = 256e-3
n_t = 31720
gradient = np.concatenate(
    [lte for i in range(107)] + [ste for i in range(107)], axis=0)
dt = T / (gradient.shape[1] - 1)
Rs = [utils.vec2vec_rotmat(np.array([1., 0, 0]), i) if np.linalg.norm(i) != 0 
      else np.eye(3) for i in bvecs]
gradient = gradients.rotate_gradient(gradient, Rs)
gradient, dt = gradients.interpolate_gradient(gradient, dt, n_t)
gradient = gradients.set_b(gradient, dt, bvals)

In [ ]:
# Run simulation with TDE1

substrate = {'type' : 'mesh',
             'mesh' : mesh,
             'initial positions' : pos[0:int(1e6)],
             'periodic' : True,
             'N_sv' : 50}
signals_1 = simulations.simulation(
    int(1e6), diffusivity, gradient, dt, substrate, all_signals=True, seed=1)

substrate = {'type' : 'mesh',
             'mesh' : mesh,
             'initial positions' : pos[int(1e6):int(2e6)],
             'periodic' : True,
             'N_sv' : 50}
signals_2 = simulations.simulation(
    int(1e6), diffusivity, gradient, dt, substrate, all_signals=True, seed=2)

substrate = {'type' : 'mesh',
             'mesh' : mesh,
             'initial positions' : pos[int(2e6)::],
             'periodic' : True,
             'N_sv' : 50}
signals_3 = simulations.simulation(
    int(1e6), diffusivity, gradient, dt, substrate, all_signals=True, seed=3)

np.save(os.path.join(path, 'simulated_signals_2.npy'),
        np.hstack((signals_1, signals_2, signals_3)))

# Simulation 3

In [ ]:
# Define waveforms

lte = np.zeros((504, 3))
lte[1:3, 0] = 1
lte[101:103, 0] = -1
lte[201:203, 0] = 1
lte[301:303, 0] = -1
lte[401:403, 0] = 1
lte[501:503, 0] = -1
lte = lte[np.newaxis, :, :]
ste = np.zeros((504, 3))
ste[1:3, 0] = 1
ste[101:103, 0] = -1
ste[201:203, 1] = 1
ste[301:303, 1] = -1
ste[401:403, 2] = 1
ste[501:503, 2] = -1
ste = ste[np.newaxis, :, :]
plt.plot(lte[0])
plt.show()
plt.plot(ste[0])
plt.show()


# Create gradient array

T = 503e-3
n_t = 62195
gradient = np.concatenate(
    [lte for i in range(107)] + [ste for i in range(107)], axis=0)
dt = T / (gradient.shape[1] - 1)
Rs = [utils.vec2vec_rotmat(np.array([1., 0, 0]), i) if np.linalg.norm(i) != 0 
      else np.eye(3) for i in bvecs]
gradient = gradients.rotate_gradient(gradient, Rs)
gradient, dt = gradients.interpolate_gradient(gradient, dt, n_t)
gradient = gradients.set_b(gradient, dt, bvals)

In [ ]:
# Run simulation with TDE2

substrate = {'type' : 'mesh',
             'mesh' : mesh,
             'initial positions' : pos[0:int(7.5e5)],
             'periodic' : True,
             'N_sv' : 50}
signals_1 = simulations.simulation(
    int(7.5e5), diffusivity, gradient, dt, substrate, all_signals=True, seed=1)

substrate = {'type' : 'mesh',
             'mesh' : mesh,
             'initial positions' : pos[int(7.5e5):int(15e5)],
             'periodic' : True,
             'N_sv' : 50}
signals_2 = simulations.simulation(
    int(7.5e5), diffusivity, gradient, dt, substrate, all_signals=True, seed=2)

substrate = {'type' : 'mesh',
             'mesh' : mesh,
             'initial positions' : pos[int(15e5):int(22.5e5)],
             'periodic' : True,
             'N_sv' : 50}
signals_3 = simulations.simulation(
    int(7.5e5), diffusivity, gradient, dt, substrate, all_signals=True, seed=3)

substrate = {'type' : 'mesh',
             'mesh' : mesh,
             'initial positions' : pos[int(22.5e5)::],
             'periodic' : True,
             'N_sv' : 50}
signals_4 = simulations.simulation(
    int(7.5e5), diffusivity, gradient, dt, substrate, all_signals=True, seed=4)


np.save(os.path.join(path, 'simulated_signals_3.npy'),
        np.hstack((signals_1, signals_2, signals_3, signals_4)))